In [12]:
import requests
import json
import base64
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
from datetime import datetime, timedelta
import pymysql
import pandas as pd
import numpy as np

In [2]:
admin_id = "hloge"
admin_pw = "gksthf95@"

client_id = "a67AZkWFGFivDkRaVYdIQD"
client_secret = "eVeeCuVVmJVk1L4WVZOVJB"
service_key = "BqpWMBpRUjjWtAznIHNU6D8dMLEO4pkcW96G5dexvh4="

mall_id = "hloge"
encode_csrf_token = "CP2" # 임의로 정하기
redirect_uri = "https://hloge.cafe24.com"

In [3]:
# 통합 API 추출
def call_total_api():
    # code 가져오기
    def get_cafe24_code(scope):
        # code를 가져와야하는 URL
        Request_URL = f"https://{mall_id}.cafe24api.com/api/v2/oauth/authorize?response_type=code&client_id={client_id}&state={encode_csrf_token}&redirect_uri={redirect_uri}&scope={scope}"
    
        # 크롤링 구조
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('no-sandox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--start-maximized')
        # options.add_argument("headless") # 백그라운드 실행 (만일 창 보이게 하고 싶으면 주석 처리)

        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36")

        browser = webdriver.Chrome("./chromedriver.exe" ,options=options)

        browser.get(Request_URL)

        def finds(css_selector):
            return browser.find_elements(By.CSS_SELECTOR, css_selector) # 여러개의 아이템을 찾을 때 사용

        def find(css_selector):
            return browser.find_element(By.CSS_SELECTOR, css_selector) # 한 개의 아이템을 찾을 때 사용

        def finds_xpath(xpath):
            return browser.find_elements(By.XPATH, xpath) # Xpath 찾을 떄 사용 (여러개)

        def find_xpath(xpath):
            return browser.find_element(By.XPATH, xpath) # Xpath 찾을 떄 사용 (한 개)

        input_id = find('input#mall_id') # 
        input_pw = find('input#userpasswd')

        pyperclip.copy(admin_id) # Copy - Control + c
        input_id.send_keys(Keys.CONTROL, "v") # Paste - Control + v

        pyperclip.copy(admin_pw) # Copy - Control + c
        input_pw.send_keys(Keys.CONTROL, "v") # Paste - Control + v
        input_pw.send_keys("\n") # Enter

        time.sleep(1.5) # 요소 생성까지 대기 시간

        # after_pw = find('a#iptBtnEm') # 비밀번호 다음에 변경하기 page
        after_pw = find_xpath('//*[@id="iptBtnEm"]')

        if after_pw: # 만일 '비밀번호 다음에 변경하기' 페이지가 나오면
            after_pw.click()
            browser.implicitly_wait(10)
            try :
                find_xpath('/html/body/div/div[3]/button[2]').click()
                time.sleep(4)
                code_url = browser.current_url # 현재 URL 추출
            except :
                code_url = browser.current_url # 현재 URL 추출
        else:
            code_url = browser.current_url # 현재 URL 추출
            
        browser.quit()

        return code_url


    # Access Token
    def get_cafe24_access(code_url):
        # 넣은 url에서 코드 따기
        code = code_url.split("=")[1].split("&")[0]

        # Authorization에 필요한 정보 base64 형식으로 인코딩
        origin_auth = f"{client_id}:{client_secret}"
        encode_bytes = origin_auth.encode('utf-8')
        encode_Data = base64.b64encode(encode_bytes)
        base64_String = encode_Data.decode('utf-8')

        # Access Token 요청하기
        access_url = f"https://{mall_id}.cafe24api.com/api/v2/oauth/token"
        payload = f'''grant_type=authorization_code&code={code}&redirect_uri={redirect_uri}'''
        access_headers = {
            'Authorization': f"Basic {base64_String}",
            'Content-Type': "application/x-www-form-urlencoded"
            }
        access_response = requests.request("POST", access_url, data=payload, headers=access_headers)
        access_token = json.loads(access_response.text)['access_token']

        return access_token

    def get_cafe24_api_common_func(access_token, want_api, offset):
        api_url = f"https://{mall_id}.cafe24api.com/api/v2/admin/{want_api}"

        # header 및 param 설정
        api_headers = {
            'Authorization': f"Bearer {access_token}",
            'Content-Type': "application/json",
            }
        order_params = {
            'offset': offset,
            'limit': 100
        }
        api_response = requests.request("GET", api_url, headers=api_headers, params=order_params)
        api_json = json.loads(api_response.text)

        return api_json


    # Order API 함수
    def get_cafe24_order_api(access_token, start_date, end_date):
        api_url = f"https://{mall_id}.cafe24api.com/api/v2/admin/orders?embed=items,return,cancellation,exchange"

        # header 및 param 설정
        api_headers = {
            'Authorization': f"Bearer {access_token}",
            'Content-Type': "application/json",
            }
        order_params = {
            'start_date': str(start_date),
            'end_date': str(end_date),
            'limit': 1000
        }
        api_response = requests.request("GET", api_url, headers=api_headers, params=order_params)
        api_json = json.loads(api_response.text)

        return api_json


    # Categories API 추출
    def call_category_api():
        cate_code_url = get_cafe24_code(scope='mall.read_category')
        cate_access_token = get_cafe24_access(code_url=cate_code_url)
        cate_api = get_cafe24_api_common_func(access_token=cate_access_token, want_api='categories', offset=0)

        return cate_api


    # Products API 추출
    def call_product_api():
        prod_code_url = get_cafe24_code(scope='mall.read_product')
        prod_access_token = get_cafe24_access(code_url=prod_code_url)
        prod_api = get_cafe24_api_common_func(access_token=prod_access_token, want_api='products', offset=0)
        prod_api_2 = get_cafe24_api_common_func(access_token=prod_access_token, want_api='products', offset=100)

        for key, value in prod_api_2.items():
            if key in prod_api:
                prod_api[key].extend(value)
            else:
                prod_api[key] = value

        return prod_api


    # Coupons API 추출
    def call_coupon_api():
        cou_code_url = get_cafe24_code(scope='mall.read_promotion')
        cou_access_token = get_cafe24_access(code_url=cou_code_url)
        cou_api = get_cafe24_api_common_func(access_token=cou_access_token, want_api='coupons', offset=0)

        return cou_api


    # Orders API 추출
    def call_order_api():
        # Order 기간 설정 (최대 3개월)
        # 1st period
        now = datetime.now()
        before = now - timedelta(days=89)

        use_now = str(now)[:10]
        use_before = str(before)[:10]

        # 2nd period
        sec_end = before - timedelta(days=1)
        sec_start = sec_end - timedelta(days=89)

        use_sec_end = str(sec_end)[:10]
        use_sec_start = str(sec_start)[:10]

        # 3rd period
        thr_end = sec_start - timedelta(days=1)
        thr_start = thr_end - timedelta(days=89)

        use_thr_end = str(thr_end)[:10]
        use_thr_start = str(thr_start)[:10]

        # 4th period
        for_end = thr_start - timedelta(days=1)
        for_start = for_end - timedelta(days=89)

        use_for_end = str(for_end)[:10]
        use_for_start = str(for_start)[:10]


        order_code_url = get_cafe24_code('mall.read_order')
        order_access_token = get_cafe24_access(order_code_url)
        order_api = get_cafe24_order_api(order_access_token, use_before, use_now)
        order_api_2 = get_cafe24_order_api(order_access_token, use_sec_start, use_sec_end)
        order_api_3 = get_cafe24_order_api(order_access_token, use_thr_start, use_thr_end)
        order_api_4 = get_cafe24_order_api(order_access_token, use_for_start, use_for_end)

        for key, value in order_api_2.items():
            if key in order_api:
                order_api[key].extend(value)
            else:
                order_api[key] = value

        for key, value in order_api_3.items():
            if key in order_api:
                order_api[key].extend(value)
            else:
                order_api[key] = value

        for key, value in order_api_4.items():
            if key in order_api:
                order_api[key].extend(value)
            else:
                order_api[key] = value

        return order_api
    
    total_api = call_category_api().copy()
    total_api.update(call_product_api())
    total_api.update(call_order_api())
    total_api.update(call_coupon_api())

    return total_api

In [4]:
total_api = call_total_api()

C:\Users\user\AppData\Local\Temp\ipykernel_21328\3406438162.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("./chromedriver.exe" ,options=options)


In [5]:
total_api

{'categories': [{'shop_no': 1,
   'category_no': 49,
   'category_depth': 1,
   'parent_category_no': 1,
   'category_name': 'All',
   'display_type': 'A',
   'full_category_name': {'1': 'All', '2': None, '3': None, '4': None},
   'full_category_no': {'1': 49, '2': None, '3': None, '4': None},
   'root_category_no': 49,
   'use_main': 'T',
   'use_display': 'T',
   'display_order': 0,
   'soldout_product_display': 'N',
   'sub_category_product_display': 'T',
   'hashtag_product_display': None,
   'hash_tags': [],
   'product_display_scope': 'A',
   'product_display_type': 'A',
   'product_display_key': 'R',
   'product_display_sort': 'D',
   'product_display_period': 'W',
   'normal_product_display_type': None,
   'normal_product_display_key': None,
   'normal_product_display_sort': None,
   'normal_product_display_period': None,
   'recommend_product_display_type': None,
   'recommend_product_display_key': None,
   'recommend_product_display_sort': None,
   'recommend_product_display_

In [6]:
total_api.keys()

dict_keys(['categories', 'products', 'orders', 'error', 'coupons'])

In [8]:
print(f"count categories : {len(total_api['categories'])}")
print(f"count products : {len(total_api['products'])}")
print(f"count orders : {len(total_api['orders'])}")
print(f"count coupons : {len(total_api['coupons'])}")

count categories : 23
count products : 119
count orders : 103
count coupons : 5


In [43]:
category_no_list = []
category_depth_list = []
parent_category_no_list = []
category_name_list = []
display_type_list = []
full_category_name_1_list = []
full_category_name_2_list = []
full_category_name_3_list = []
full_category_name_4_list = []
full_category_no_1_list = []
full_category_no_2_list = []
full_category_no_3_list = []
full_category_no_4_list = []
root_category_no_list = []
use_main_list = []
display_order_list = []


for i in range(len(total_api['categories'])) :
    category_no = total_api['categories'][i]['category_no']
    category_depth = total_api['categories'][i]['category_depth']
    parent_category_no = total_api['categories'][i]['parent_category_no']
    category_name = total_api['categories'][i]['category_name']
    display_type = total_api['categories'][i]['display_type']
    full_category_name_1 = total_api['categories'][i]['full_category_name']['1']
    full_category_name_2 = total_api['categories'][i]['full_category_name']['2']
    full_category_name_3 = total_api['categories'][i]['full_category_name']['3']
    full_category_name_4 = total_api['categories'][i]['full_category_name']['4']
    full_category_no_1 = total_api['categories'][i]['full_category_no']['1']
    full_category_no_2 = total_api['categories'][i]['full_category_no']['2']
    full_category_no_3 = total_api['categories'][i]['full_category_no']['3']
    full_category_no_4 = total_api['categories'][i]['full_category_no']['4']
    root_category_no = total_api['categories'][i]['root_category_no']
    use_main = total_api['categories'][i]['use_main']
    display_order = total_api['categories'][i]['display_order']

    category_no_list.append(category_no)
    category_depth_list.append(category_depth)
    parent_category_no_list.append(parent_category_no)
    category_name_list.append(category_name)
    display_type_list.append(display_type)
    full_category_name_1_list.append(full_category_name_1)
    full_category_name_2_list.append(full_category_name_2)
    full_category_name_3_list.append(full_category_name_3)
    full_category_name_4_list.append(full_category_name_4)
    full_category_no_1_list.append(full_category_no_1)
    full_category_no_2_list.append(full_category_no_2)
    full_category_no_3_list.append(full_category_no_3)
    full_category_no_4_list.append(full_category_no_4)
    root_category_no_list.append(root_category_no)
    use_main_list.append(use_main)
    display_order_list.append(display_order)

In [88]:
category_dic = {'분류 번호':category_no_list,
                '분류 깊이':category_depth_list,
                '부모 분류 번호':parent_category_no_list,
                '분류명':category_name_list,
                '쇼핑몰 표시설정':display_type_list,
                '대분류명':full_category_name_1_list,
                '중분류명':full_category_name_2_list,
                '소분류명':full_category_name_3_list,
                '세분류명':full_category_name_4_list,
                '대분류 번호':full_category_no_1_list,
                '중분류 번호':full_category_no_2_list,
                '소분류 번호':full_category_no_3_list,
                '세분류 번호':full_category_no_4_list,
                '최상위 분류 번호':root_category_no_list,
                '메인분류 표시상태':use_main_list,
                '진열 순서':display_order_list}

In [337]:
category_df = pd.DataFrame(category_dic)

In [338]:
for i in range(7, 13):
    category_df['분류명'][i] = category_df['분류명'][i].split(") ")[1]
    category_df['소분류명'][i] = category_df['소분류명'][i].split(") ")[1]

for i in [8,9,11,12]:
    category_df['세분류명'][i] = category_df['세분류명'][i].split(") ")[1]

category_df[['중분류 번호', '소분류 번호', '세분류 번호']] = category_df[['중분류 번호', '소분류 번호', '세분류 번호']].fillna(0).astype('int').replace(0, None)

C:\Users\user\AppData\Local\Temp\ipykernel_21328\3776639831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_df['분류명'][i] = category_df['분류명'][i].split(") ")[1]
C:\Users\user\AppData\Local\Temp\ipykernel_21328\3776639831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_df['소분류명'][i] = category_df['소분류명'][i].split(") ")[1]
C:\Users\user\AppData\Local\Temp\ipykernel_21328\3776639831.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [339]:
category_df

,분류 번호,분류 깊이,부모 분류 번호,분류명,쇼핑몰 표시설정,대분류명,중분류명,소분류명,세분류명,대분류 번호,중분류 번호,소분류 번호,세분류 번호,최상위 분류 번호,메인분류 표시상태,진열 순서
0,49,1,1,All,A,All,None,None,None,49,None,None,None,49,T,0
1,24,1,1,Outerwear,A,Outerwear,None,None,None,24,None,None,None,24,T,1
2,29,2,24,Jacket,A,Outerwear,Jacket,None,None,24,29,None,None,24,F,2
3,30,2,24,Coat,A,Outerwear,Coat,None,None,24,30,None,None,24,F,3
4,31,2,24,Cardigan,A,Outerwear,Cardigan,None,None,24,31,None,None,24,F,4
5,25,1,1,Top,A,Top,None,None,None,25,None,None,None,25,T,5
6,32,2,25,Tee,A,Top,Tee,None,None,25,32,None,None,25,F,6
7,36,3,32,Short Sleeve,A,Top,Tee,Short Sleeve,None,25,32,36,None,25,F,7
8,38,4,36,Cropped,A,Top,Tee,Short Sleeve,Cropped,25,32,36,38,25,F,8
9,39,4,36,Graphic Tees,A,Top,Tee,Short Sleeve,Graphic Tees,25,32,36,39,25,F,9


In [253]:
product_no_list = []
product_code_list = []
prod_category_no_list = []
product_name_list = []
price_excluding_tax_list = []
price_list = []
retail_price_list = []
supply_price_list = []
display_list = []
selling_list = []
product_condition_list = []
created_date_list = []
sold_out_list = []

for i in range(len(total_api['products'])) :
    product_no = total_api['products'][i]['product_no']
    product_code = total_api['products'][i]['product_code']
    product_name = total_api['products'][i]['product_name']
    price_excluding_tax = total_api['products'][i]['price_excluding_tax']
    price = total_api['products'][i]['price']
    retail_price = total_api['products'][i]['retail_price']
    supply_price = total_api['products'][i]['supply_price']
    display = total_api['products'][i]['display']
    selling = total_api['products'][i]['selling']
    product_condition = total_api['products'][i]['product_condition']
    created_date = total_api['products'][i]['created_date'][:10]
    sold_out = total_api['products'][i]['sold_out']

    product_no_list.append(product_no)
    product_code_list.append(product_code)
    product_name_list.append(product_name)
    price_excluding_tax_list.append(price_excluding_tax)
    price_list.append(price)
    retail_price_list.append(retail_price)
    supply_price_list.append(supply_price)
    display_list.append(display)
    selling_list.append(selling)
    product_condition_list.append(product_condition)
    created_date_list.append(created_date)
    sold_out_list.append(sold_out)

In [147]:
# code 가져오기
def get_cafe24_code(scope):
    # code를 가져와야하는 URL
    Request_URL = f"https://{mall_id}.cafe24api.com/api/v2/oauth/authorize?response_type=code&client_id={client_id}&state={encode_csrf_token}&redirect_uri={redirect_uri}&scope={scope}"
   
    # 크롤링 구조
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('no-sandox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--start-maximized')
    # options.add_argument("headless") # 백그라운드 실행 (만일 창 보이게 하고 싶으면 주석 처리)

    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36")

    browser = webdriver.Chrome("./chromedriver.exe" ,options=options)

    browser.get(Request_URL)

    def finds(css_selector):
        return browser.find_elements(By.CSS_SELECTOR, css_selector) # 여러개의 아이템을 찾을 때 사용

    def find(css_selector):
        return browser.find_element(By.CSS_SELECTOR, css_selector) # 한 개의 아이템을 찾을 때 사용

    def finds_xpath(xpath):
        return browser.find_elements(By.XPATH, xpath) # Xpath 찾을 떄 사용 (여러개)

    def find_xpath(xpath):
        return browser.find_element(By.XPATH, xpath) # Xpath 찾을 떄 사용 (한 개)

    input_id = find('input#mall_id') # 
    input_pw = find('input#userpasswd')

    pyperclip.copy(admin_id) # Copy - Control + c
    input_id.send_keys(Keys.CONTROL, "v") # Paste - Control + v

    pyperclip.copy(admin_pw) # Copy - Control + c
    input_pw.send_keys(Keys.CONTROL, "v") # Paste - Control + v
    input_pw.send_keys("\n") # Enter

    time.sleep(1.5) # 요소 생성까지 대기 시간

    # after_pw = find('a#iptBtnEm') # 비밀번호 다음에 변경하기 page
    after_pw = find_xpath('//*[@id="iptBtnEm"]')

    if after_pw: # 만일 '비밀번호 다음에 변경하기' 페이지가 나오면
        after_pw.click()
        browser.implicitly_wait(10)
        try :
            find_xpath('/html/body/div/div[3]/button[2]').click()
            time.sleep(4)
            code_url = browser.current_url # 현재 URL 추출
        except :
            code_url = browser.current_url # 현재 URL 추출
    else:
        code_url = browser.current_url # 현재 URL 추출
        
    browser.quit()

    return code_url

In [148]:
# Access Token
def get_cafe24_access(code_url):
    # 넣은 url에서 코드 따기
    code = code_url.split("=")[1].split("&")[0]

    # Authorization에 필요한 정보 base64 형식으로 인코딩
    origin_auth = f"{client_id}:{client_secret}"
    encode_bytes = origin_auth.encode('utf-8')
    encode_Data = base64.b64encode(encode_bytes)
    base64_String = encode_Data.decode('utf-8')

    # Access Token 요청하기
    access_url = f"https://{mall_id}.cafe24api.com/api/v2/oauth/token"
    payload = f'''grant_type=authorization_code&code={code}&redirect_uri={redirect_uri}'''
    access_headers = {
        'Authorization': f"Basic {base64_String}",
        'Content-Type': "application/x-www-form-urlencoded"
        }
    access_response = requests.request("POST", access_url, data=payload, headers=access_headers)
    access_token = json.loads(access_response.text)['access_token']

    return access_token

In [252]:
# Products API 함수
def get_cafe24_api_product_func(access_token, number):
    api_url = f"https://{mall_id}.cafe24api.com/api/v2/admin/products/{number}"

    # header 및 param 설정
    api_headers = {
        'Authorization': f"Bearer {access_token}",
        'Content-Type': "application/json",
        }
    api_response = requests.request("GET", api_url, headers=api_headers)
    api_json = json.loads(api_response.text)

    return api_json

In [239]:
prod_code_url = get_cafe24_code(scope='mall.read_product')
prod_access_token = get_cafe24_access(code_url=prod_code_url)

C:\Users\user\AppData\Local\Temp\ipykernel_21328\2194318857.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("./chromedriver.exe" ,options=options)


In [254]:
for i in product_no_list :
    prod_category_no = get_cafe24_api_product_func(prod_access_token, i)['product']['category'][0]['category_no']
    prod_category_no_list.append(prod_category_no)
    time.sleep(0.5)

In [257]:
product_dic = {'상품 번호':product_no_list,
               '상품 코드':product_code_list,
               '분류 번호':prod_category_no_list,
               '상품명':product_name_list, 
               '상품가(세금 제외)':price_excluding_tax_list, 
               '상품 판매가':price_list,
               '상품 소비자가':retail_price_list, 
               '상품 공급가':supply_price_list, 
               '진열 상태':display_list,
               '판매 상태':selling_list, 
               '상품 상태':product_condition_list, 
               '생성일':created_date_list, 
               '품절 여부':sold_out_list}

In [260]:
product_df = pd.DataFrame(product_dic)

In [273]:
product_df[['상품가(세금 제외)', '상품 판매가', '상품 소비자가', '상품 공급가']] = product_df[['상품가(세금 제외)', '상품 판매가', '상품 소비자가', '상품 공급가']].astype('float').astype('int')

In [274]:
product_df

,상품 번호,상품 코드,분류 번호,상품명,상품가(세금 제외),상품 판매가,상품 소비자가,상품 공급가,진열 상태,판매 상태,상품 상태,생성일,품절 여부
0,143,P00000FN,42,애니멀 오버핏 니트 (2color),35318,38850,0,21000,T,T,N,2023-01-05,F
1,142,P00000FM,26,러브 오프숄더 미니 원피스 (2color),26909,29600,0,16000,T,T,N,2023-01-03,F
2,141,P00000FL,47,폼폼 메리제인 플랫 (3color),25455,28000,0,14000,T,T,N,2022-12-30,F
3,140,P00000FK,35,언발 하이웨스트 투웨이 치마바지 (2color),22000,24200,0,11000,T,T,N,2022-12-30,F
4,139,P00000FJ,33,크롭 셔링 퍼프 글램 팝 블라우스 (4color),39455,43400,0,28000,T,T,N,2022-12-20,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,17,P000000R,48,(여리여리/데이트룩/페미닌) 트임 카라 크롭 스트라이프 니트 투피스 (3color),30909,34000,0,17000,T,T,N,2022-05-01,F
115,15,P000000P,33,(최저가!/쫀쫀/편함/고퀄) 체크 셔링 퍼프 벌룬 스퀘어 스모크 블라우스 셔츠 (3...,36182,39800,0,20000,T,T,N,2022-05-01,F
116,13,P000000N,36,베어 크롭 반팔티 (2color),18000,19800,0,0,T,T,N,2022-04-30,F
117,12,P000000M,26,아가엘 pk 니트 반팔 원피스,24364,26800,28000,0,T,T,N,2022-04-30,F


In [354]:
order_id_list = []
order_product_no_list = []
order_product_code_list = []
order_benefit_price_list = []
member_id_list = []
member_email_list = []
payment_method_name_list = []
paid_list = []
canceled_list = []
order_date_list = []
first_order_list = []
order_from_mobile_list = []
initial_order_amount_list = []
actual_order_amount_list = []
payment_amount_list = []
order_place_name_list = []
quantity_list = []

for i in range(len(total_api['orders'])) :
    order_id = total_api['orders'][i]['order_id']
    order_product_no = total_api['orders'][i]['items'][0]['product_no']
    order_product_code = total_api['orders'][i]['items'][0]['product_code']
    order_benefit_price = total_api['orders'][i]['actual_order_amount']['coupon_discount_price']
    member_id = total_api['orders'][i]['member_id']
    member_email = total_api['orders'][i]['member_email']
    payment_method_name = total_api['orders'][i]['payment_method_name'][0]
    paid = total_api['orders'][i]['paid']
    canceled = total_api['orders'][i]['canceled']
    order_date = total_api['orders'][i]['order_date'][:10]
    first_order = total_api['orders'][i]['first_order']
    order_from_mobile = total_api['orders'][i]['order_from_mobile']
    initial_order_amount = total_api['orders'][i]['initial_order_amount']['order_price_amount']
    actual_order_amount = total_api['orders'][i]['actual_order_amount']['order_price_amount']
    payment_amount = total_api['orders'][i]['payment_amount']
    order_place_name = total_api['orders'][i]['order_place_name']
    quantity = total_api['orders'][i]['items'][0]['quantity']

    order_id_list.append(order_id)
    order_product_no_list.append(order_product_no)
    order_product_code_list.append(order_product_code)
    order_benefit_price_list.append(order_benefit_price)
    member_id_list.append(member_id)
    member_email_list.append(member_email)
    payment_method_name_list.append(payment_method_name)
    paid_list.append(paid)
    canceled_list.append(canceled)
    order_date_list.append(order_date)
    first_order_list.append(first_order)
    order_from_mobile_list.append(order_from_mobile)
    initial_order_amount_list.append(initial_order_amount)
    actual_order_amount_list.append(actual_order_amount)
    payment_amount_list.append(payment_amount)
    order_place_name_list.append(order_place_name)
    quantity_list.append(quantity)

In [356]:
order_dic = {'주문 번호':order_id_list,
             '상품 번호':order_product_no_list,
             '상품 코드':order_product_code_list,
             '쿠폰 할인 금액':order_benefit_price_list,
             '회원 아이디':member_id_list,
             '회원 이메일':member_id_list,
             '결제수단명':payment_method_name_list,
             '결제 여부':paid_list,
             '취소 여부':canceled_list,
             '주문일':order_date_list,
             '최초 주문여부':first_order_list,
             '모바일 구분':order_from_mobile_list,
             '최초 주문 금액':initial_order_amount_list,
             '현재 주문 금액':actual_order_amount_list,
             '최종 결제 금액':payment_amount_list,
             '주문경로 이름':order_place_name_list,
             '주문 상품 갯수':quantity_list}

In [357]:
order_df = pd.DataFrame(order_dic)

In [358]:
order_df[['회원 아이디', '회원 이메일']] = order_df[['회원 아이디', '회원 이메일']].replace('', np.NaN)
order_df[['쿠폰 할인 금액', '최초 주문 금액', '현재 주문 금액', '최종 결제 금액']] = order_df[['쿠폰 할인 금액', '최초 주문 금액', '현재 주문 금액', '최종 결제 금액']].astype('float').astype('int')

In [359]:
order_df

,주문 번호,상품 번호,상품 코드,쿠폰 할인 금액,회원 아이디,회원 이메일,결제수단명,결제 여부,취소 여부,주문일,최초 주문여부,모바일 구분,최초 주문 금액,현재 주문 금액,최종 결제 금액,주문경로 이름,주문 상품 갯수
0,20230130-0000036,141,P00000FL,0,NaN,NaN,기타,T,F,2023-01-30,F,F,28000,28000,28000,에이블리,1
1,20230130-0000026,131,P00000FB,0,2643157625@k,2643157625@k,신용카드,T,F,2023-01-30,T,T,26900,26900,29900,모바일웹,1
2,20230125-0000010,109,P00000EF,0,NaN,NaN,신용카드,T,F,2023-01-25,F,F,0,0,32800,지그재그,1
3,20230116-0000013,118,P00000EO,0,NaN,NaN,기타,T,T,2023-01-16,F,F,14850,0,0,에이블리,1
4,20230112-0000019,99,P00000DV,0,NaN,NaN,신용카드,T,T,2023-01-12,F,F,29500,0,0,지그재그,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20220525-0000017,62,P00000CK,0,NaN,NaN,기타,T,F,2022-05-25,F,F,15700,15700,15700,에이블리,1
99,20220518-0000018,58,P00000CG,0,NaN,NaN,기타,T,F,2022-05-18,F,F,48000,48000,48000,에이블리,1
100,20220516-0000016,23,P000000X,0,NaN,NaN,기타,T,F,2022-05-16,F,F,32000,32000,32000,에이블리,1
101,20220511-0000015,40,P00000BO,0,NaN,NaN,신용카드,T,F,2022-05-11,F,F,39800,39800,38420,스마트스토어,1


In [364]:
coupon_no_list= []
benefit_price_list= []
coupon_type_list= []
coupon_name_list= []
created_date_list= []
deleted_list= []
benefit_text_list= []
benefit_percentage_list= []
issue_member_join_list= []
issued_count_list= []

for i in range(len(total_api['coupons'])) :
    coupon_no = total_api['coupons'][i]['coupon_no']
    benefit_price = total_api['coupons'][i]['benefit_price']
    coupon_type = total_api['coupons'][i]['coupon_type']
    coupon_name = total_api['coupons'][i]['coupon_name']
    created_date = total_api['coupons'][i]['created_date'][:10]
    deleted = total_api['coupons'][i]['deleted']
    benefit_text = total_api['coupons'][i]['benefit_text']
    benefit_percentage = total_api['coupons'][i]['benefit_percentage']
    issue_member_join = total_api['coupons'][i]['issue_member_join']
    issued_count = total_api['coupons'][i]['issued_count']

    coupon_no_list.append(coupon_no)
    benefit_price_list.append(benefit_price)
    coupon_type_list.append(coupon_type)
    coupon_name_list.append(coupon_name)
    created_date_list.append(created_date)
    deleted_list.append(deleted)
    benefit_text_list.append(benefit_text)
    benefit_percentage_list.append(benefit_percentage)
    issue_member_join_list.append(issue_member_join)
    issued_count_list.append(issued_count)

In [365]:
coupon_dic = {'쿠폰 번호':coupon_no_list,
              '쿠폰 할인 금액':benefit_price_list,
              '쿠폰 유형':coupon_type_list,
              '쿠폰명':coupon_name_list,
              '생성일':created_date_list,
              '쿠폰삭제 여부':deleted_list,
              '쿠폰혜택 상세내역':benefit_text_list,
              '혜택 비율':benefit_percentage_list,
              '회원가입시 쿠폰 발급 여부':issue_member_join_list,
              '발급된 수량':issued_count_list}

In [366]:
coupon_df = pd.DataFrame(coupon_dic)                                    

In [369]:
coupon_df['쿠폰 할인 금액'] = coupon_df['쿠폰 할인 금액'].fillna(0).astype('float').astype('int').replace(0, None)

In [370]:
coupon_df

,쿠폰 번호,쿠폰 할인 금액,쿠폰 유형,쿠폰명,생성일,쿠폰삭제 여부,쿠폰혜택 상세내역,혜택 비율,회원가입시 쿠폰 발급 여부,발급된 수량
0,6072901192800000316,7000,O,신규 가입 회원 200000원 이상 구매시 7000원 할인 적용,2022-06-23,F,7000원 할인,None,T,6
1,6072901163500000315,5000,O,신규 가입 회원 100000원 이상구매시 5000원 할인 적용♡,2022-06-23,F,5000원 할인,None,T,6
2,6072901155300000314,None,O,신규 가입 회원 70000원 이상 구매시 무료배송 적용♡,2022-06-23,F,전액 할인,None,T,6
3,6072901150200000313,1000,O,신규 가입 회원 50000원 이상구매시 1000원 할인 적용♡,2022-06-23,F,1000원 할인,None,T,6
4,6072901137400000312,3000,O,"신규 가입 회원 3,000원 할인♡",2022-06-23,F,3000원 할인,None,T,6


In [9]:
host = 'donone-db.cyfp8pevwfx6.ap-northeast-1.rds.amazonaws.com'
user = 'root'
password = '&donone1!K&'
db = 'dononedb'

In [10]:
# DB 연결
conn = pymysql.connect(host=host, user=user, password=password, db=db, charset='utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)

In [ ]:
# DB 저장
conn.commit()

In [ ]:
# 연결 해제
conn.close()